In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from copy import deepcopy

from symmetries.analysis.lie_symmetry_analysis import point_symmetries
from symmetries.utils.symbolic import sym_det_eqn

from symmetries.objects.determining_equations import DeterminingEquations
import sympy as sp
from sympy import Derivative as D

# Function to propose a general form of the infinitesimals.

### Continuity Equation

In [4]:
x, y, z, t = sp.symbols('x y z t')
rho = sp.Function('rho')(x, y, z, t)
f = sp.Function('f')(x, y, z, t)
h = sp.Function('h')(x, y, z, t)
B = sp.Function('B')(x, y, z, t)
P = sp.Function('P')(x, y, z, t)
independent_variables = [x, y, z, t]
dependent_variables = [f, h, rho, B, P]
constants = []
F =  D(rho,t) + rho*D(f,z) + f*D(rho,z)
F_rules_array = {D(rho,t):-rho*D(f,z)-f*D(rho,z)}
# F =  rho*D(v,t) + D(P,y) + B*D(B,z)
# F_rules_array = {D(P,y):-rho*D(v,t) - B*D(B,z)}
# F = v*D(B,z) + B*D(h,y) + h*D(B,y) + B*D(f,x) + f*D(B,x)
# F_rules_array = {D(B,t):-B*D(h,y) - h*D(B,y) - B*D(f,x) - f*D(B,x)}
# F = v*D(B,z) + B*D(v,z)
# F_rules_array = {D(B,z): B*v*D(v,z)}
order = 1
F.expand()

f(x, y, z, t)*Derivative(rho(x, y, z, t), z) + rho(x, y, z, t)*Derivative(f(x, y, z, t), z) + Derivative(rho(x, y, z, t), t)

In [6]:
system_of_equations = point_symmetries(F, order, F_rules_array, independent_variables, dependent_variables, constants)

deleting xix B(x, y, z, t)
deleting xiy B(x, y, z, t)
deleting xix P(x, y, z, t)
deleting xiy P(x, y, z, t)
deleting xix h(x, y, z, t)
deleting xiy h(x, y, z, t)
deleting xit B(x, y, z, t)
deleting xit P(x, y, z, t)
deleting xit h(x, y, z, t)
deleting xix f(x, y, z, t)
deleting xix rho(x, y, z, t)
deleting xiy f(x, y, z, t)
deleting xiy rho(x, y, z, t)
deleting xix z
deleting xiy z
deleting etarho B(x, y, z, t)
deleting etarho P(x, y, z, t)
deleting etarho h(x, y, z, t)
found deleted variable in xit eq 0
found deleted variable in xit eq 1
found deleted variable in xit eq 3
found deleted variable in xit eq 4
found deleted variable in xit eq 5
found deleted variable in xit eq 7
found deleted variable in xit eq 8
found deleted variable in xit eq 11
found deleted variable in xix eq 15
found deleted variable in xix eq 15
found deleted variable in xiy eq 16
found deleted variable in xiy eq 16
found deleted variable in xit eq 17
found deleted variable in etarho eq 46
found deleted variable in

In [ ]:
system_of_equations.model.differential_equation

B*f_x + B*h_y + B_t + B_x*f + B_y*h

In [7]:
system_of_equations.print_determining_equations()

Matrix([
[ 0,                                                                                                                                                                                            Eq(xi^(z)_B, 0)],
[ 1,                                                                                                                                                                                            Eq(xi^(z)_P, 0)],
[ 2,                                                                                                       Eq(xi^(t)_f*f(x, y, z, t)**2 - xi^(t)_rho*f(x, y, z, t)*rho(x, y, z, t) - xi^(z)_f*f(x, y, z, t), 0)],
[ 3,                                                                                                                                                                                            Eq(xi^(z)_h, 0)],
[ 4,                                                                                                   Eq(xi^(t)_f*f(x, y, z, t)*rho(x, y, z, t) - xi^(

In [ ]:
system_of_equations.print_latex()

-1B^2\xi^{t}_{B}+1fB\xi^{t}_{f}-1B\xi^{x}_{f}=0\\
-2B^2\xi^{t}_{B}+1fB\xi^{t}_{f}+1hB\xi^{t}_{h}=0\\
-1B^2\xi^{t}_{B}+1hB\xi^{t}_{h}-1B\xi^{y}_{h}=0\\
-1fB\xi^{t}_{B}+1f^2\xi^{t}_{f}-1f\xi^{x}_{f}=0\\
-1fB\xi^{t}_{B}+1B\xi^{x}_{B}+1fh\xi^{t}_{h}-1h\xi^{x}_{h}=0\\
1fB\xi^{t}_{h}-1B\xi^{x}_{h}=0\\
1fB\xi^{t}_{h}-1B\xi^{x}_{h}=0\\
-1hB\xi^{t}_{B}+1B\xi^{y}_{B}+1fh\xi^{t}_{f}-1f\xi^{y}_{f}=0\\
-1hB\xi^{t}_{B}+1h^2\xi^{t}_{h}-1h\xi^{y}_{h}=0\\
1hB\xi^{t}_{f}-1B\xi^{y}_{f}=0\\
1hB\xi^{t}_{f}-1B\xi^{y}_{f}=0\\
-1B\xi^{t}_{B}+1f\xi^{t}_{f}-1\xi^{x}_{f}=0\\
-1B\xi^{y}_{B}+1fh\xi^{t}_{f}-1h\xi^{x}_{f}=0\\
\xi^{z}_{B}=0\\
-1B\xi^{t}_{B}+1h\xi^{t}_{h}-1\xi^{y}_{h}=0\\
-1B\xi^{x}_{B}+1fh\xi^{t}_{h}-1f\xi^{y}_{h}=0\\
\xi^{z}_{f}=0\\
\xi^{z}_{h}=0\\
\xi^{t}_{h}=0\\
\xi^{t}_{f}=0\\
-1B\xi^{x}_{f}-1B\xi^{y}_{h}=0\\
\xi^{x}_{h}=0\\
\xi^{y}_{f}=0\\
1fB\xi^{t}_{x}+1hB\xi^{t}_{y}-1B\eta^{B}_{B}+1B\eta^{f}_{f}+1B\xi^{t}_{t}-1B\xi^{x}_{x}+1\eta^{B}}+1f\eta^{B}_{f}=0\\
1fB\xi^{t}_{x}+1hB\xi^{t}_{y}-1B\eta^{B}

In [ ]:
x, t, k = sp.symbols('x t k')
u = sp.Function('u')(x, t)
independent_variables = [x, t]
dependent_variables = [u]
constants = [k]
F =  D(u,t) - k*D(u,x,x)
F_rules_array = {D(u,x,x):1/k*D(u,t)}
order = 2
F.expand()

-k*Derivative(u(x, t), (x, 2)) + Derivative(u(x, t), t)

In [ ]:
system_of_equations = point_symmetries(F, order, F_rules_array, independent_variables, dependent_variables, constants)

In [ ]:
system_of_equations.print_determining_equations()

Matrix([
[0,                                 Eq(xi^(t)_u_u, 0)],
[1,                                 Eq(xi^(x)_u_u, 0)],
[2,                                   Eq(xi^(t)_u, 0)],
[3,            Eq(-eta^(u)_u_u*k + 2*k*xi^(x)_x_u, 0)],
[4,                Eq(2*k*xi^(t)_x_u + 2*xi^(x)_u, 0)],
[5,                                   Eq(xi^(t)_x, 0)],
[6,       Eq(k*xi^(t)_x_x - xi^(t)_t + 2*xi^(x)_x, 0)],
[7, Eq(-2*eta^(u)_x_u*k + k*xi^(x)_x_x - xi^(x)_t, 0)],
[8,                  Eq(eta^(u)_t - eta^(u)_x_x*k, 0)]])